In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, DepthwiseConv2D, BatchNormalization, AveragePooling2D, Dense, Activation, Dropout, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping


#fixing seed
tensorflow.random.set_seed(0)
np.random.seed(0)

In [ ]:
direc = r'/content/drive/My Drive/bci_iv_2a_data/Original/' #original data directory
generated_data_dir = r'/content/drive/My Drive/bci_iv_2a_data/Generated/' #generated data directory

mms = MinMaxScaler() #minmaxscaler

train_dataset = []
train_label = []

generated_train_dataset = []
generated_train_label = []

test_dataset = []
test_label = []


# loading original EEG training dataset
for f, folder in enumerate(os.listdir(direc)):
    train_folder = glob.glob(direc + folder + '/train/')
    for i in range(0,len(os.listdir(train_folder[0]))):
        files = os.listdir(train_folder[0]+str(i))
        for j, name in enumerate(files):
            filename = glob.glob(train_folder[0]+str(i)+'/'+name)
            df = pd.read_csv(filename[0], index_col=None, header=None)
            df = df.drop(0, axis=1) #dropping column of eeg channel names
            df = df.iloc[:,0:1000] #taking 1000 timesteps
            df = pd.DataFrame(mms.fit_transform(df)) #min-max scaling
            train_dataset.append(np.array(df))
            train_label.append(i) #appending class labels
            

# loading original EEG test dataset
for f, folder in enumerate(os.listdir(direc)):
    test_folder = glob.glob(direc + folder + '/test/')
    for i in range(0,len(os.listdir(test_folder[0]))):
        files = os.listdir(test_folder[0]+str(i))
        for j, name in enumerate(files):
            filename = glob.glob(test_folder[0]+str(i)+'/'+name)
            df = pd.read_csv(filename[0], index_col=None, header=None)
            df = df.drop(0, axis=1) #dropping column of eeg channel names
            df = df.iloc[:,0:1000] #taking 1000 timesteps
            df = pd.DataFrame(mms.fit_transform(df)) #min-max scaling
            test_dataset.append(np.array(df)) 
            test_label.append(i) #appending class labels
            

# loading generated EEG dataset
for f, folder in enumerate(os.listdir(generated_data_dir)):
    train_folder = glob.glob(generated_data_dir + folder + '/train/')
    for i in range(0,len(os.listdir(train_folder[0]))):
        files = os.listdir(train_folder[0]+str(i))
        for j, name in enumerate(files):
            filename = glob.glob(train_folder[0]+str(i)+'/'+name)
            df = pd.read_csv(filename[0], index_col=None, header=None)
            df = df.iloc[:,0:1000] #taking 1000 timesteps
            df = pd.DataFrame(mms.fit_transform(df)) #min-max scaling
            generated_train_dataset.append(np.array(df))
            generated_train_label.append(i) #appending class labels


In [ ]:
train_dataset = np.array(train_dataset)
train_label = np.array(train_label)

generated_train_dataset = np.array(generated_train_dataset)
generated_train_label = np.array(generated_train_label)

test_dataset = np.array(test_dataset)
test_label = np.array(test_label)


In [ ]:
train_dataset = np.expand_dims(train_dataset,axis=-1)
train_label = to_categorical(train_label)

generated_train_dataset = np.expand_dims(generated_train_dataset,axis=-1)
generated_train_label = to_categorical(generated_train_label)

test_dataset = np.expand_dims(test_dataset,axis=-1)

In [ ]:
# EEGNet architecture
def EEGNet(nb_classes, Channels = 64, Samples = 128, dropout_rate = 0.5, kernelLength = 64, F1 = 8, D = 2, F2 = 16):
    
    """     
      Ref: Lawhern, Vernon & Solon, Amelia & Waytowich, Nicholas & Gordon, Stephen & Hung, Chou & Lance, Brent. (2016). 
      EEGNet: A Compact Convolutional Network for EEG-based Brain-Computer Interfaces. 
      Journal of Neural Engineering. 15. 10.1088/1741-2552/aace8c. 

      nb_classes         : number of classes to classify
      Channels, Samples  : number of channels and time steps 
      dropout_rate       : dropout rate
      kernelLength       : length of temporal convolution in first layer.    
      F1, F2             : number of temporal filters (F1) and number of pointwise filters (F2).  
      D                  : number of spatial filters
    """
    
    
    inputs   = Input(shape = (Channels, Samples, 1))

    block1       = Conv2D(F1, (1, kernelLength), padding = 'same', input_shape = (Channels, Samples, 1))(inputs)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Channels, 1), depth_multiplier = D, depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = Dropout(dropout_rate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16), padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = Dropout(dropout_rate)(block2)
        
    flatten      = Flatten()(block2)
    
    dense        = Dense(nb_classes)(flatten)
    softmax      = Activation('softmax')(dense)
    
    model = Model(inputs=inputs, outputs=softmax)
    model.summary()
    
    return model

In [ ]:
#specifying EEGNet for both training with both original and generated dataset
eegnet_original_data = eegnet_generated_data = EEGNet(4, Channels = 22, Samples = 500, dropout_rate = 0.2, kernelLength = 64, F1 = 8,  D = 2, F2 = 16)

eegnet_original_data.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics= ['accuracy'])
eegnet_generated_data.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics= ['accuracy'])


In [ ]:
# early stopping callback
callbacks = EarlyStopping(monitor = 'val_accuracy',
                          mode='min',
                          patience = 30,
                          verbose = 1,
                          restore_best_weights = True)

In [ ]:
# training with original data
X_train, X_test, y_train, y_test = train_test_split(train_dataset,train_label, test_size=0.2)

history1  = eegnet_original_data.fit(X_train, y_train, epochs=150, validation_data = (X_test, y_test))

In [ ]:
# training with generated data
X_train2, X_test2, y_train2, y_test2 = train_test_split(generated_train_dataset,generated_train_label, test_size=0.2)

history2  = eegnet_generated_data.fit(X_train2, y_train2, epochs=150, validation_data = (X_test2, y_test2))

In [ ]:
#loss curves
loss1 = history1.history['loss']
loss2 = history2.history['loss']
epochs = range(1, len(loss1) + 1)
plt.figure(figsize=(7,5))
plt.plot(epochs, loss1, 'r', label='loss with original training data')
plt.plot(epochs, loss2, 'b', label='loss with generated training data')
plt.legend()
plt.show()


In [ ]:
#classification report with original training data and test data
print(classification_report(test_label, np.argmax(eegnet_original_data.predict(test_dataset), axis = 1)))


In [ ]:
#classification report with generated training data and test data
print(classification_report(test_label, np.argmax(eegnet_generated_data.predict(test_dataset), axis = 1)))
